In [11]:
import os
import openai
import sys
# sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.environ['API_KEY']

In [3]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import torch
from collections import Counter

# === 加载和构建时相同的嵌入模型 ===
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-zh-v1.5",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

# === 加载本地 FAISS 数据库 ===
index_path = "./faiss_law_index"
vectordb = FAISS.load_local(
    index_path,
    embedding_model,
    allow_dangerous_deserialization=True
)

# === 获取所有文档片段 ===
all_docs = list(vectordb.docstore._dict.values())

# 总片段数
print(f"📊 总文档片段数量: {len(all_docs)}")

# 统计每个文件的片段数
file_counter = Counter(doc.metadata.get("source", "未知文件") for doc in all_docs)

print("\n📂 向量库包含的文件及片段数：")
for file_name, count in file_counter.items():
    print(f"  {file_name}: {count} 个片段")

# 列出所有文件名
print("\n📄 文件列表：")
for file_name in file_counter.keys():
    print(f"  {file_name}")


📊 总文档片段数量: 597

📂 向量库包含的文件及片段数：
  福田区物业城市泥头车道路污染“七有”全流程治理手册20250424（修订稿）.docx: 46 个片段
  03泥头车治理上位规范0522/泥头车道路污染事件分类分级清单20250625V2.xlsx: 18 个片段
  03泥头车治理上位规范0522/深圳市建筑废弃物管理办法.pdf: 125 个片段
  03泥头车治理上位规范0522/附件18《福田区城市环境卫生规范与标准》.pdf: 197 个片段
  03泥头车治理上位规范0522/泥头车资料来源.xlsx: 3 个片段
  03泥头车治理上位规范0522/泥头车运输标准执行表20250513(1).xlsx: 5 个片段
  03泥头车治理上位规范0522/公共区域环境卫生质量和管理要求DB4403.pdf: 73 个片段
  01业务流程图/泥头车七有网格20250324.xlsx: 1 个片段
  02泥头车部门职责素材/[已签章附件]关于印发《福田区综合网格事项管理办法（试行）》的通知.pdf: 85 个片段
  02泥头车部门职责素材/20250109 福田区泥头车遗撒污染路面问题“七有”网格化工作指引（征求意见稿）.docx: 16 个片段
  02泥头车部门职责素材/[福田管理局发文] 区泥头车整治办关于印发《福田区建筑废弃物运输行业安全生产专项整治工作方案》的通知 419 17_37.pdf: 18 个片段
  02泥头车部门职责素材/深圳市生态环境保护委员会办公室关于加强渣土运输车辆扬尘污染专项整治的补充通知.pdf: 4 个片段
  02泥头车部门职责素材/关于泥头车执法的各部门职责分工及法律依据.xlsx: 6 个片段

📄 文件列表：
  福田区物业城市泥头车道路污染“七有”全流程治理手册20250424（修订稿）.docx
  03泥头车治理上位规范0522/泥头车道路污染事件分类分级清单20250625V2.xlsx
  03泥头车治理上位规范0522/深圳市建筑废弃物管理办法.pdf
  03泥头车治理上位规范0522/附件18《福田区城市环境卫生规范与标准》.pdf
  03泥头车治理上位规范0522/泥头车资料来源.xlsx
  03泥头车治理上位规范0522/泥头车运输标准执行表20250513(1).x

In [4]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import torch

# 重新加载嵌入模型（需和构建时一致）
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-zh-v1.5",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

# 加载本地FAISS索引
index_path = "./faiss_law_index"
vectordb = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True)

# 查询示例
query = "泥头车道路遗洒，城市管家应该做什么？相关法律法规是什么"
results = vectordb.similarity_search(query, k=10)  # k是返回的条目数

# 输出结果及对应文件名
print(f"🔎 Query: {query}\n")
for i, res in enumerate(results, 1):
    print(f"结果 {i}:")
    print(f"  来源文件: {res.metadata.get('source', '未知文件')}")
    print(f"  内容: {res.page_content[:100]}...\n")


🔎 Query: 泥头车道路遗洒，城市管家应该做什么？相关法律法规是什么

结果 1:
  来源文件: 03泥头车治理上位规范0522/泥头车道路污染事件分类分级清单20250625V2.xlsx
  内容: 市政服务第三方企业人员：在日常辅道区域保洁时，若发现泥头车遗撒污染道路等情况；以及在日常水渠绿化管养等过程中，发现余泥渣土乱倾倒问题，及时拍摄记录车牌号及遗撒证据，上报城市管家。 巡查内容与规范\n遗...

结果 2:
  来源文件: 福田区物业城市泥头车道路污染“七有”全流程治理手册20250424（修订稿）.docx
  内容: 3.配合协助。在规定的业务范围和区域内，调度工作人员协助执法人员依法开展泥头车遗撒问题专项整治行动，辅助管理等综合执法工作。

社区网格员

1.信息上报。网格员若在日常巡查以及居民诉求的途径发现泥头...

结果 3:
  来源文件: 03泥头车治理上位规范0522/泥头车道路污染事件分类分级清单20250625V2.xlsx
  内容: 街道巡查员/社区网格员：工地每日巡查频次≥2次。\n 物业城市服务单位 城市管家 城市管家：负责日常主车道区域巡逻。 巡查内容与规范\n遗撒监控：重点检查主车道是否存在泥浆、砂石等散落物，核查是否为泥...

结果 4:
  来源文件: 福田区物业城市泥头车道路污染“七有”全流程治理手册20250424（修订稿）.docx
  内容: 市政服务第三方企业

（一）环卫工人：负责直接清理遗撒物，使用铲子、铁锹、扫帚等工具铲除或清扫路面硬化泥块、渣土等。

（二）驾驶员：驾驶洗扫车、洒水车、高压水车等设备，对路面进行冲刷软化或冲洗。

...

结果 5:
  来源文件: 03泥头车治理上位规范0522/泥头车道路污染事件分类分级清单20250625V2.xlsx
  内容: 监督主体（监督市政服务第三方企业） 城管局 量化考核指标 构建市政服务第三方企业考核加分、员工激励制度。 （一）企业考核加分：\n上报有效信息，纳入政府购买服务年度考核指标，直接影响合同续签优先级。\...

结果 6:
  来源文件: 福田区物业城市泥头车道路污染“七有”全流程治理手册20250424（修订稿）.docx
  内容: 管理圈层（发现）

（一）街道巡查员/社区网格员：负责日常工地巡查。

（二）城市管家：

/home/fk/miniconda3/envs/holistic/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [17]:
from langchain.chat_models import ChatOpenAI  # 可替换为 deepseek-r1
from langchain.schema import HumanMessage
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

#  加载 FAISS 向量库
vectordb = FAISS.load_local(
    folder_path="./faiss_law_index",  # 你保存的知识库路径
    embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-large-zh-v1.5"),
    allow_dangerous_deserialization=True
)

#  构造检索函数
def query_law_index(query_text: str, k=3):
    docs = vectordb.similarity_search(query_text, k=k)
    return "\n\n".join([doc.page_content for doc in docs])

#  构造 LangChain 模型（可替换为 DeepSeek）
# llm = ChatOpenAI(temperature=0, model="gpt-4")  # 或调用 deepseek-r1 的兼容封装

llm = ChatOpenAI(
    model="r1w8a8",
    temperature=0,
    openai_api_key=os.environ["W8A8_API_KEY"],
    openai_api_base="http://223.2.249.70:7019/v1",
)

#  模拟状态输入
state = {
    'filled_input': '运输企业在运输过程中，存在泥头车路面遗洒，责任如何划分？'
}

#  查询法规知识库
rag_context = query_law_index(state['filled_input'])

#  构造最终 Prompt
prompt = f"""
你是一位城市治理法规专家，请结合以下法规内容判断城市泥头车污染事件的责任归属。

法规内容如下：
{rag_context}

描述如下：
{state['filled_input']}

请判断责任归属，并说明理由。
输出 JSON 格式：
{{
  "responsibility": "运输企业",
  "reason": "......"
}}
"""

#  调用大模型进行回答
response = llm.invoke([HumanMessage(content=prompt)])

#  输出结果
print("📘 模型返回：")
print(response.content)


/home/fk/miniconda3/envs/holistic/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


📘 模型返回：
<think>
好的，我现在需要处理这个问题，判断城市泥头车污染事件的责任归属。根据提供的法规内容，用户问的是运输企业在运输过程中存在泥头车路面遗洒，责任如何划分。首先，我需要仔细阅读并理解相关的法规条款，然后分析责任归属。

首先，法规中关于市政服务第三方企业的部分提到，环卫工人负责清理遗撒物，驾驶员负责使用车辆冲洗路面，而泥头车遗撒的处置费用由肇事运输企业追缴。这说明运输企业如果导致遗撒，需要承担费用，可能涉及责任。

接下来是执法圈层主体与职责，街道办事处负责巡查和综合执法，包括发现泥头车的问题，比如泄漏、遗撒等，并约谈涉事企业和工地。交警部门则核查运输路线，处理超载、撒漏等违法行为。这部分说明执法部门有监管责任，但具体执行问题可能还是由运输企业负责。

自治圈层考核标准中的运输企业部分，正面清单要求合法运营、车辆安全规范、安全运输等。而负面清单提到如果车辆未安装密闭装置导致遗撒，或者驾驶员无证、超载、不按路线行驶，运输企业会被追责。特别是第3点明确，如果运输过程中有超载、不按指定路线等行为，运输企业要负责。此外，法规还提到如果运输企业未建立信息化管理，也可能被追责。

工地企业的责任包括选择合规的运输企业，并监督他们的运输过程。如果工地没有做好出入口的冲洗设施，导致车辆带泥上路，工地也要负责。但在这个案例中，问题出在运输过程中，而不是工地的出入口管理，所以可能工地责任较小。

用户描述的情况是运输企业在运输过程中出现遗洒，所以需要看运输企业是否违反了相关规定。根据运输企业的负面清单，如果车辆未安装密闭装置，或者驾驶员违规操作，比如超载、不按路线行驶，运输企业应承担责任。此外，法规明确提到遗撒处置费用由运输企业追缴，这也支持责任在运输企业。

因此，责任应归属于运输企业，因为他们没有遵守车辆安全规范，比如密闭装置的问题，或者驾驶员未按指定路线行驶，导致遗撒发生。同时，法规中直接指出遗撒费用由运输企业承担，进一步确认了他们的责任。
</think>

```json
{
  "responsibility": "运输企业",
  "reason": "根据法规中关于运输企业的负面清单规定，若因未安装密闭装置导致物料遗撒，或存在驾驶员无证驾驶、超载、未按指定路线运输等违规行为，运输企业需承担直接责任。同时，法规明确要求泥头车遗撒处置费用应向肇事运输